In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from settings import *

## Read Data

In [290]:
con, meta = connectToDB('smeros', '', 'sciArticles')

sql = 'select id, body, retweet_count, publishing_date \
from document \
-- where doc_type = \'twitter\' \
'

documents = pd.read_sql_query(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
documents.set_index('id')

,body,retweet_count,publishing_date
id,,,
db896cca-4b4f-11e5-8a0f-42010af074fc,The School Nutrition Association’s bizarre opp...,NaN,2015-03-10 15:15:37
e159e47c-4b4f-11e5-8bbc-42010af074fc,Domino's Pizza manager T.J. Djigue creates a h...,NaN,2012-06-19 00:00:00
e2531dd0-4b4f-11e5-90b6-42010af074fc,People less interested in food might be at a h...,NaN,2012-06-25 00:00:00
f5520f9a-4b4f-11e5-9ba0-42010af074fc,Obesity is a global epidemic. The World Health...,NaN,2012-07-11 00:00:00
f5bc2ac4-4b4f-11e5-821c-42010af074fc,"A proposed ban on super-sized, sugary sodas in...",NaN,2012-07-23 17:52:27
f6a0d61a-4b4f-11e5-8e22-42010af074fc,"Producers\n\nIn Search Of A Drought Strategy, ...",NaN,2015-08-19 00:00:00
f727ea60-4b4f-11e5-8e62-42010af074fc,When it comes to the crop insurance sweepstake...,NaN,2012-04-12 21:29:07
faec8ade-4b4f-11e5-b537-42010af074fc,The U.S. Department of Agriculture got into a ...,NaN,2012-07-26 13:12:16
fc6304ec-4b4f-11e5-8769-42010af074fc,"Contrary to conventional wisdom, detailed ener...",NaN,2012-07-26 00:00:00


In [291]:
def extractLinks(documents):
    import re
    from urllib.request import urlopen
    from urllib.parse import urlparse
    from urllib.error import HTTPError

    urlRegex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    urls = documents['body'].apply(lambda x: re.findall(urlRegex, x))
    urls=urls.rename('urls')
    
    documents = pd.DataFrame(pd.concat([documents['id'], urls], axis=1))
    documents.set_index('id')
    
    return documents
documents = extractLinks(documents.head(50))

In [292]:
def countLinks(docs):
    print(docs)
    docs['urls'] = docs['urls'].apply(lambda x: len(x))
    a = docs.groupby('urls').size()
    print (a)

countLinks(documents)

                                      id                                               urls
0   db896cca-4b4f-11e5-8a0f-42010af074fc                                                 []
1   e159e47c-4b4f-11e5-8bbc-42010af074fc                                                 []
2   e2531dd0-4b4f-11e5-90b6-42010af074fc                                                 []
3   f5520f9a-4b4f-11e5-9ba0-42010af074fc                                                 []
4   f5bc2ac4-4b4f-11e5-821c-42010af074fc                                                 []
5   f6a0d61a-4b4f-11e5-8e22-42010af074fc                                                 []
6   f727ea60-4b4f-11e5-8e62-42010af074fc                                                 []
7   faec8ade-4b4f-11e5-b537-42010af074fc                                                 []
8   fc6304ec-4b4f-11e5-8769-42010af074fc                                                 []
9   07b5e7c2-4d05-11e5-8778-42010af074fc                                        

In [236]:
def flattenLinks(documents):
    
    #documentsWithUrl = documents.where(documents['urls'].apply(lambda x: len(x)!=0)).dropna(how='all')
    #documentsWithoutUrl = documents.where(documents['urls'].apply(lambda x: len(x)==0))['id'].dropna(how='all')
    
    l = pd.DataFrame([(row[1], link) for row in documents.itertuples() for link in row[2]])
    print (l)
        
    
documents1 = flattenLinks(documents.head(50))

                                      0                                                  1
0  0edffff8-4b50-11e5-b3ba-42010af074fc  http://farmdoc.illinois.edu/marketing/weekly/h...
1  6d03d226-4b50-11e5-8769-42010af074fc                    https://twitter.com/DHochsprung
2  6d03d226-4b50-11e5-8769-42010af074fc                               http://bit.ly/QYEbhO
3  655d9994-4b50-11e5-821c-42010af074fc                              http://usat.ly/S0G7bb
4  88a75a0c-4b50-11e5-821c-42010af074fc                             http://usat.ly/10RHcR4


In [211]:
                                                
#     documents = pd.DataFrame(pd.concat([documents['id'], urls], axis=1))
#     documents.set_index('id')
        
    
#     def test(x):
#         d = {}
#         for url in x['urls']:
#             d(x['id']) = url
    
#     a = documents.apply(lambda x : test(x), axis=1)
    
    
    
    
#     print(a)
    
#     print(documents)
    
    def resolveURLs(urls):

        if (removeDuplicateLinks):
            urls = set(urls)
            
        resolvedURLs = pd.DataFrame
        unresolvedURLs = pd.DataFrame

        for url in urls:
            try:
                resolvedURL = urlopen(url,timeout=10).geturl()
                
                if(urlparse(resolvedURL).netloc.endswith('.ly')):
                    print('not resolved: ' + url)
                    unresolvedURLs.append(url)
                    continue
                
                print ('resolved: ' + resolvedURL)
                resolvedURLs.append(resolvedURL)
            except HTTPError as e:
                print (e.code)
                print('not resolved: ' + url)
                resolvedURLs.append('DeadLink')
        #return [session.head(url, allow_redirects=True).url for url in urls]
    
    

    
    
    #urls = urls.apply(lambda x: resolveURLs(x))
    
    print(urls.shape)
    
    #Checks resolvability of URLs
    #resolvableURLs = urls.apply(pd.Series).applymap(lambda x: None if pd.isnull(x) else requests.get(x, verify=False).status_code==200)
    #resolvableURLs = pd.Series(np.logical_or.reduce(resolvableURLs, axis=1), name="hasResolvableURLs")
    #print (resolvableURLs.shape)


    return documents

documents1 = extractURLs(documents.head(50))

IndentationError: unexpected indent (<ipython-input-211-c3f4131f2fcd>, line 20)

In [194]:


#tweets1 = extractURLs(tweets)

print (tweets1['urls'][0:10])

#conn.setopt(pycurl.URL, "http://t.co/yzUy11usmg")
#conn.setopt(pycurl.URL, "http://t.co/cEVKs7ewVN")

NameError: name 'tweets1' is not defined

In [195]:
print (urls)

NameError: name 'urls' is not defined